In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, when, rand, current_date, date_add, date_format, udf
from pyspark.sql.types import StringType
import os
import json

spark = SparkSession.builder.appName("SyntheticData") \
    .config("spark.master", "local") \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

sc = spark.sparkContext

# Указываем сколько строк надо сгенерировать
numbers_row = 100

# Генерация данных для колонок id, name, city, email, age, salary, registration_date
data = [(i, f"Names_{i}", f"Cityname_{i}") for i in range(1, numbers_row + 1)]
df = spark.createDataFrame(data, ["id", "name", "city"])

df = df.withColumn("email", concat(col("name"), lit("@example."), when(col("id") % 2 == 0, "ru").otherwise("com"))) \
    .withColumn("age", (rand() * 78 + 18).cast("int")) \
    .withColumn("salary", (rand() * 150000 + 19242).cast("int")) \
    .withColumn("registration_date", date_add(current_date(), - (rand() * (col("age") - 18)).cast("int") * 365))

df.cache()  # Кеширование DataFrame для более быстрой обработки

# Замена 5% данных на значение NULL
columns = ["name", "email", "city", "age", "salary", "registration_date"]
df = df.select("id", *[when(rand() <= 0.05, None).otherwise(col(column)).alias(column) for column in columns])

# Открываем словарь с алгритмом шифрования
with open('/home/jovyan/work/PySpark_test/General_task_№2/utils/encryption_dict.json', 'r') as f:
    encryption_dict = json.load(f)

# Отправляем на узлы кластера
broadcast_dict = sc.broadcast(encryption_dict).value

# Функция для шифрования
def encrypt_string(input_string):
    if input_string is None:
        return None
    return ''.join(broadcast_dict.get(char, char) for char in str(input_string))

# Определяем UDF для запуска функции шифрования
encrypt_udf = udf(encrypt_string, StringType())

# Применяем UDF к колоннам
df = df.withColumn("name", encrypt_udf(col("name"))) \
    .withColumn("email", encrypt_udf(col("email"))) \
    .withColumn("city", encrypt_udf(col("city"))) \
    .withColumn("age", encrypt_udf(col("age"))) \
    .withColumn("salary", encrypt_udf(col("salary"))) \
    .withColumn("registration_date", encrypt_udf(col("registration_date")))

# Пишу свое имя на первой строке колонки name
df = df.withColumn("name", when(col("id") == 1, "ATAGAEV").otherwise(col("name")))

# Установка текущей даты и количества строк для имени файла
df_date = spark.sql("SELECT current_date()")
current_date = df_date.select(date_format("current_date", "yyyy-MM-dd")).first()[0]
name_csv = encrypt_string(numbers_row)

# Путь к сохранению CSV файла
path = f"/home/jovyan/work/PySpark_test/config/temp/{current_date}-{name_csv}.csv"
path_rename = f"/home/jovyan/work/PySpark_test/config/{current_date}-{name_csv}.csv"
# Запись DataFrame в CSV
df.coalesce(1).write.csv(path, header=True, mode="overwrite")  

# Переименование сгенерированного файла
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    if file.startswith("part-"):
        os.rename(file_path, path_rename)

# Удаление временных файлов и папки
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    os.remove(file_path)
    
os.rmdir(path)

df.show()

spark.stop()

+---+--------------------+--------------------+--------------------+------------+--------------------+--------------------+
| id|                name|               email|                city|         age|              salary|   registration_date|
+---+--------------------+--------------------+--------------------+------------+--------------------+--------------------+
|  1|             ATAGAEV|78nbc2dff41nak647...|vbn345967nd34sf33...| eight8zero1|one1three1four4fo...|two1zero1one1seve...|
|  2|78nbc2dff41nak647...|78nbc2dff41nak647...|vbn345967nd34sf33...| five5three1|one1two1eight8six...|two1zero1one1zero...|
|  3|78nbc2dff41nak647...|78nbc2dff41nak647...|vbn345967nd34sf33...|   six6nine9|four4two1six6one1...|two1zero1two1zero...|
|  4|78nbc2dff41nak647...|78nbc2dff41nak647...|vbn345967nd34sf33...|    six6two1|nine9two1four4eig...|one1nine9eight8tw...|
|  5|78nbc2dff41nak647...|78nbc2dff41nak647...|vbn345967nd34sf33...| eight8five5|four4zero1two1eig...|two1zero1zero1two...|
|  6|78n

In [2]:
##########################################################################
# Блок кода для дешифрования
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, current_date, date_format
from pyspark.sql.types import StringType
import json
import re
import os

# Инициализация Spark
spark = SparkSession.builder.appName("SyntheticData") \
    .config("spark.master", "local") \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

sc = spark.sparkContext

# Указываем сколько было строк в зашифрованном файле
numbers_row = 100

# Открытие словаря шифрования
with open('/home/jovyan/work/PySpark_test/General_task_№2/utils/encryption_dict.json', 'r') as f:
    encryption_dict = json.load(f)
    
# Отправляем на узлы кластера
broadcast_dict = sc.broadcast(encryption_dict).value

# Функция для шифрования
def encrypt_string(input_string):
    if input_string is None:
        return None
    return ''.join(broadcast_dict.get(char, char) for char in str(input_string))

# Установка текущей даты и количества строк для имени файла
df_date = spark.sql("SELECT current_date()")
current_date = df_date.select(date_format("current_date", "yyyy-MM-dd")).first()[0]
name_csv = encrypt_string(numbers_row)
path_open = f"/home/jovyan/work/PySpark_test/config/{current_date}-{name_csv}.csv"

# Чтение DataFrame из CSV
df = spark.read.csv(path_open, header=True)

# Создание обратного словаря
decryption_dict = {v: k for k, v in broadcast_dict.items()}

# Функция для дешифрования
def decrypt_string(encrypted):
    if encrypted is None:
        return None 
    decrypted = []
    pattern = '|'.join(re.escape(key) for key in decryption_dict.keys())
    matches = re.findall(pattern, encrypted)
    if not matches:
        return encrypted
    for match in matches:
        decrypted.append(decryption_dict.get(match, match))
    return ''.join(decrypted)

# Определяем UDF для дешифрования
decrypt_udf = udf(decrypt_string, StringType())

# Применяем UDF к нужным колоннам
df = df.withColumn("name", decrypt_udf(col("name"))) \
    .withColumn("email", decrypt_udf(col("email"))) \
    .withColumn("city", decrypt_udf(col("city"))) \
    .withColumn("age", decrypt_udf(col("age"))) \
    .withColumn("salary", decrypt_udf(col("salary"))) \
    .withColumn("registration_date", decrypt_udf(col("registration_date")))

# Путь к сохранению CSV файла
path = f"/home/jovyan/work/PySpark_test/config/temp/{current_date}-dev.csv"
path_rename = f"/home/jovyan/work/PySpark_test/config/{current_date}-dev.csv"
# Запись DataFrame в CSV
df.coalesce(1).write.csv(path, header=True, mode="overwrite")  

# Переименование сгенерированного файла
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    if file.startswith("part-"):
        os.rename(file_path, path_rename)

# Удаление временных файлов и папки
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    os.remove(file_path)
    
os.rmdir(path)

df.show()

spark.stop()

+---+--------+--------------------+-----------+---+------+-----------------+
| id|    name|               email|       city|age|salary|registration_date|
+---+--------+--------------------+-----------+---+------+-----------------+
|  1| ATAGAEV| Names_1@example.com| Cityname_1| 80|134472|       2017-08-03|
|  2| Names_2|  Names_2@example.ru| Cityname_2| 53|128621|       2010-08-05|
|  3| Names_3| Names_3@example.com| Cityname_3| 69| 42612|       2020-08-02|
|  4| Names_4|  Names_4@example.ru| Cityname_4| 62| 92485|       1982-08-12|
|  5| Names_5| Names_5@example.com| Cityname_5| 85| 40284|       2002-08-07|
|  6| Names_6|  Names_6@example.ru| Cityname_6| 24| 60038|       2020-08-02|
|  7| Names_7| Names_7@example.com| Cityname_7| 42| 93294|       2006-08-06|
|  8| Names_8|  Names_8@example.ru| Cityname_8| 80|119254|       2003-08-07|
|  9| Names_9| Names_9@example.com| Cityname_9| 74| 32157|       1976-08-13|
| 10|Names_10| Names_10@example.ru|Cityname_10| 78|157504|       2020-08-02|